In [1]:
import pandas as pd
from tensorflow.keras.applications.efficientnet import EfficientNetB4, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [7]:
BATCH_SIZE = 64
LEARNING_RATE = 0.001 # Adam's default = 0.001; SGD's default = 0.01 
IMAGE_SIZE = (380, 380)
INPUT_SHAPE = (380, 380, 3)
EPOCHS = 5

In [3]:
train_df = pd.read_csv('output/img_balanced.csv')
train_df = train_df[['filepath', 'category']]
train_df['category'] = train_df['category'].astype(str)

In [4]:
# efficientnet = EfficientNetB4(include_top=True, weights='imagenet')
# efficientnet.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 380, 380, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 380, 380, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 380, 380, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 380, 380, 3)  0           ['normalization[0][0

 block2a_expand_conv (Conv2D)   (None, 190, 190, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 190, 190, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 190, 190, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 191, 191, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

 ivation)                                                                                         
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 95, 95, 192)  1728       ['block2c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2c_bn (BatchNormalization  (None, 95, 95, 192)  768        ['block2c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2c_activation (Activation  (None, 95, 95, 192)  0          ['block2c_bn[0][0]']             
 )                                                                                                
          

 block3a_dwconv (DepthwiseConv2  (None, 48, 48, 192)  4800       ['block3a_dwconv_pad[0][0]']     
 D)                                                                                               
                                                                                                  
 block3a_bn (BatchNormalization  (None, 48, 48, 192)  768        ['block3a_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3a_activation (Activation  (None, 48, 48, 192)  0          ['block3a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3a_se_squeeze (GlobalAver  (None, 192)         0           ['block3a_activation[0][0]']     
 agePoolin

 agePooling2D)                                                                                    
                                                                                                  
 block3c_se_reshape (Reshape)   (None, 1, 1, 336)    0           ['block3c_se_squeeze[0][0]']     
                                                                                                  
 block3c_se_reduce (Conv2D)     (None, 1, 1, 14)     4718        ['block3c_se_reshape[0][0]']     
                                                                                                  
 block3c_se_expand (Conv2D)     (None, 1, 1, 336)    5040        ['block3c_se_reduce[0][0]']      
                                                                                                  
 block3c_se_excite (Multiply)   (None, 48, 48, 336)  0           ['block3c_activation[0][0]',     
                                                                  'block3c_se_expand[0][0]']      
          

...

 block6b_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6b_se_reshape[0][0]']     
                                                                                                  
 block6b_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block6b_se_reduce[0][0]']      
                                                                                                  
 block6b_se_excite (Multiply)   (None, 12, 12, 1632  0           ['block6b_activation[0][0]',     
                                )                                 'block6b_se_expand[0][0]']      
                                                                                                  
 block6b_project_conv (Conv2D)  (None, 12, 12, 272)  443904      ['block6b_se_excite[0][0]']      
                                                                                                  
 block6b_project_bn (BatchNorma  (None, 12, 12, 272)  1088       ['block6b_project_conv[0][0]']   
 lization)

                                                                                                  
 block6d_se_excite (Multiply)   (None, 12, 12, 1632  0           ['block6d_activation[0][0]',     
                                )                                 'block6d_se_expand[0][0]']      
                                                                                                  
 block6d_project_conv (Conv2D)  (None, 12, 12, 272)  443904      ['block6d_se_excite[0][0]']      
                                                                                                  
 block6d_project_bn (BatchNorma  (None, 12, 12, 272)  1088       ['block6d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6d_drop (Dropout)         (None, 12, 12, 272)  0           ['block6d_project_bn[0][0]']     
          

                                                                                                  
 block6f_project_conv (Conv2D)  (None, 12, 12, 272)  443904      ['block6f_se_excite[0][0]']      
                                                                                                  
 block6f_project_bn (BatchNorma  (None, 12, 12, 272)  1088       ['block6f_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6f_drop (Dropout)         (None, 12, 12, 272)  0           ['block6f_project_bn[0][0]']     
                                                                                                  
 block6f_add (Add)              (None, 12, 12, 272)  0           ['block6f_drop[0][0]',           
                                                                  'block6e_add[0][0]']            
          

 block6h_project_bn (BatchNorma  (None, 12, 12, 272)  1088       ['block6h_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6h_drop (Dropout)         (None, 12, 12, 272)  0           ['block6h_project_bn[0][0]']     
                                                                                                  
 block6h_add (Add)              (None, 12, 12, 272)  0           ['block6h_drop[0][0]',           
                                                                  'block6g_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 12, 12, 1632  443904      ['block6h_add[0][0]']            
                                )                                                                 
          

 top_conv (Conv2D)              (None, 12, 12, 1792  802816      ['block7b_add[0][0]']            
                                )                                                                 
                                                                                                  
 top_bn (BatchNormalization)    (None, 12, 12, 1792  7168        ['top_conv[0][0]']               
                                )                                                                 
                                                                                                  
 top_activation (Activation)    (None, 12, 12, 1792  0           ['top_bn[0][0]']                 
                                )                                                                 
                                                                                                  
 avg_pool (GlobalAveragePooling  (None, 1792)        0           ['top_activation[0][0]']         
 2D)      

In [4]:
# instantiate a base model with pre-trained weights, exclude output layers of the model
base_model = EfficientNetB4(include_top=False, weights='imagenet')
base_model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                                3)                                                   

                                                                                                  
 block1b_add (Add)              (None, None, None,   0           ['block1b_drop[0][0]',           
                                24)                               'block1a_project_bn[0][0]']     
                                                                                                  
 block2a_expand_conv (Conv2D)   (None, None, None,   3456        ['block1b_add[0][0]']            
                                144)                                                              
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, None, None,   576        ['block2a_expand_conv[0][0]']    
 ization)                       144)                                                              
                                                                                                  
 block2a_e

                                32)                               'block2a_project_bn[0][0]']     
                                                                                                  
 block2c_expand_conv (Conv2D)   (None, None, None,   6144        ['block2b_add[0][0]']            
                                192)                                                              
                                                                                                  
 block2c_expand_bn (BatchNormal  (None, None, None,   768        ['block2c_expand_conv[0][0]']    
 ization)                       192)                                                              
                                                                                                  
 block2c_expand_activation (Act  (None, None, None,   0          ['block2c_expand_bn[0][0]']      
 ivation)                       192)                                                              
          

 block2d_add (Add)              (None, None, None,   0           ['block2d_drop[0][0]',           
                                32)                               'block2c_add[0][0]']            
                                                                                                  
 block3a_expand_conv (Conv2D)   (None, None, None,   6144        ['block2d_add[0][0]']            
                                192)                                                              
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, None, None,   768        ['block3a_expand_conv[0][0]']    
 ization)                       192)                                                              
                                                                                                  
 block3a_expand_activation (Act  (None, None, None,   0          ['block3a_expand_bn[0][0]']      
 ivation) 

                                                                                                  
 block3c_expand_conv (Conv2D)   (None, None, None,   18816       ['block3b_add[0][0]']            
                                336)                                                              
                                                                                                  
 block3c_expand_bn (BatchNormal  (None, None, None,   1344       ['block3c_expand_conv[0][0]']    
 ization)                       336)                                                              
                                                                                                  
 block3c_expand_activation (Act  (None, None, None,   0          ['block3c_expand_bn[0][0]']      
 ivation)                       336)                                                              
                                                                                                  
 block3c_d

...

 block6c_expand_bn (BatchNormal  (None, None, None,   6528       ['block6c_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6c_expand_activation (Act  (None, None, None,   0          ['block6c_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6c_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6c_expand_activation[0][0]
 D)                             1632)                            ']                               
                                                                                                  
 block6c_bn (BatchNormalization  (None, None, None,   6528       ['block6c_dwconv[0][0]']         
 )        

                                                                                                  
 block6e_expand_bn (BatchNormal  (None, None, None,   6528       ['block6e_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6e_expand_activation (Act  (None, None, None,   0          ['block6e_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6e_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6e_expand_activation[0][0]
 D)                             1632)                            ']                               
                                                                                                  
 block6e_b

                                1632)                                                             
                                                                                                  
 block6g_expand_bn (BatchNormal  (None, None, None,   6528       ['block6g_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6g_expand_activation (Act  (None, None, None,   0          ['block6g_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6g_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6g_expand_activation[0][0]
 D)                             1632)                            ']                               
          

 block7a_expand_conv (Conv2D)   (None, None, None,   443904      ['block6h_add[0][0]']            
                                1632)                                                             
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, None, None,   6528       ['block7a_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block7a_expand_activation (Act  (None, None, None,   0          ['block7a_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, None, None,   14688      ['block7a_expand_activation[0][0]
 D)       

                                                                                                  
 top_activation (Activation)    (None, None, None,   0           ['top_bn[0][0]']                 
                                1792)                                                             
                                                                                                  
Total params: 17,673,823
Trainable params: 17,548,616
Non-trainable params: 125,207
__________________________________________________________________________________________________


In [5]:
# freeze the base model
base_model.trainable=False
base_model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                                3)                                                   

                                                                                                  
 block1b_add (Add)              (None, None, None,   0           ['block1b_drop[0][0]',           
                                24)                               'block1a_project_bn[0][0]']     
                                                                                                  
 block2a_expand_conv (Conv2D)   (None, None, None,   3456        ['block1b_add[0][0]']            
                                144)                                                              
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, None, None,   576        ['block2a_expand_conv[0][0]']    
 ization)                       144)                                                              
                                                                                                  
 block2a_e

                                32)                               'block2a_project_bn[0][0]']     
                                                                                                  
 block2c_expand_conv (Conv2D)   (None, None, None,   6144        ['block2b_add[0][0]']            
                                192)                                                              
                                                                                                  
 block2c_expand_bn (BatchNormal  (None, None, None,   768        ['block2c_expand_conv[0][0]']    
 ization)                       192)                                                              
                                                                                                  
 block2c_expand_activation (Act  (None, None, None,   0          ['block2c_expand_bn[0][0]']      
 ivation)                       192)                                                              
          

 block2d_add (Add)              (None, None, None,   0           ['block2d_drop[0][0]',           
                                32)                               'block2c_add[0][0]']            
                                                                                                  
 block3a_expand_conv (Conv2D)   (None, None, None,   6144        ['block2d_add[0][0]']            
                                192)                                                              
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, None, None,   768        ['block3a_expand_conv[0][0]']    
 ization)                       192)                                                              
                                                                                                  
 block3a_expand_activation (Act  (None, None, None,   0          ['block3a_expand_bn[0][0]']      
 ivation) 

                                                                                                  
 block3c_expand_conv (Conv2D)   (None, None, None,   18816       ['block3b_add[0][0]']            
                                336)                                                              
                                                                                                  
 block3c_expand_bn (BatchNormal  (None, None, None,   1344       ['block3c_expand_conv[0][0]']    
 ization)                       336)                                                              
                                                                                                  
 block3c_expand_activation (Act  (None, None, None,   0          ['block3c_expand_bn[0][0]']      
 ivation)                       336)                                                              
                                                                                                  
 block3c_d

...

 block6c_expand_bn (BatchNormal  (None, None, None,   6528       ['block6c_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6c_expand_activation (Act  (None, None, None,   0          ['block6c_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6c_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6c_expand_activation[0][0]
 D)                             1632)                            ']                               
                                                                                                  
 block6c_bn (BatchNormalization  (None, None, None,   6528       ['block6c_dwconv[0][0]']         
 )        

                                                                                                  
 block6e_expand_bn (BatchNormal  (None, None, None,   6528       ['block6e_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6e_expand_activation (Act  (None, None, None,   0          ['block6e_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6e_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6e_expand_activation[0][0]
 D)                             1632)                            ']                               
                                                                                                  
 block6e_b

                                1632)                                                             
                                                                                                  
 block6g_expand_bn (BatchNormal  (None, None, None,   6528       ['block6g_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block6g_expand_activation (Act  (None, None, None,   0          ['block6g_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block6g_dwconv (DepthwiseConv2  (None, None, None,   40800      ['block6g_expand_activation[0][0]
 D)                             1632)                            ']                               
          

 block7a_expand_conv (Conv2D)   (None, None, None,   443904      ['block6h_add[0][0]']            
                                1632)                                                             
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, None, None,   6528       ['block7a_expand_conv[0][0]']    
 ization)                       1632)                                                             
                                                                                                  
 block7a_expand_activation (Act  (None, None, None,   0          ['block7a_expand_bn[0][0]']      
 ivation)                       1632)                                                             
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, None, None,   14688      ['block7a_expand_activation[0][0]
 D)       

                                                                                                  
 top_activation (Activation)    (None, None, None,   0           ['top_bn[0][0]']                 
                                1792)                                                             
                                                                                                  
Total params: 17,673,823
Trainable params: 0
Non-trainable params: 17,673,823
__________________________________________________________________________________________________


In [6]:
# add final layers on top of base model
input = Input(shape=INPUT_SHAPE)
x = base_model(input)
x = GlobalAveragePooling2D()(x)
x = Dropout(rate=0.2)(x)
output = Dense(units=42, activation='softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, None, None, 1792)  17673823 
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 42)                75306     
                                                                 
Total params: 17,749,129
Trainable params: 75,306
Non-trainable params: 17,673,823
____________________________________________

In [7]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])

In [8]:
def train_model(model):
    image_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input, 
        validation_split=0.2,
#         rotation_range=30,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         brightness_range=[0.8,1.2],
#         shear_range=0.2,
#         zoom_range=0.2,
#         channel_shift_range=90,
#         fill_mode='reflect',
#         horizontal_flip=True,
#         vertical_flip=False
    )

    train_generator=image_datagen.flow_from_dataframe(
        dataframe=train_df, 
        directory=None,
        x_col='filepath', 
        y_col='category', 
        class_mode='categorical', 
        color_mode='rgb',
        target_size=IMAGE_SIZE, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42,
        subset='training'
    )

    validation_generator=image_datagen.flow_from_dataframe(
        dataframe=train_df, 
        directory=None,
        x_col='filepath', 
        y_col='category', 
        class_mode='categorical', 
        color_mode='rgb',
        target_size=IMAGE_SIZE, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42,
        subset='validation'
    )

    # indices_classes_mapping = {v: k for k, v in train_generator.class_indices.items()}
    # indices_classes_df = pd.DataFrame(indices_classes_mapping.items(), columns=['index', 'class'])
    # indices_classes_df.to_csv('output/indices_classes_mapping.csv', index=False)
    
    # model_checkpoint = ModelCheckpoint('output/model/inception_resnet_{epoch:02d}.h5', 
    #                                    period=1, 
    #                                    save_weights_only=False)

    # csv_logger = CSVLogger('output/training_history.log', append=True)
    
    model.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_generator.n/train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.n/validation_generator.batch_size,
        epochs=EPOCHS,
        workers=2
    ) # callbacks=[model_checkpoint, csv_logger], use_multiprocessing=True
    
    model.save('output/models/efficientnetb4.h5')

In [9]:
train_model(model)

Found 86815 validated image filenames belonging to 42 classes.
Found 21703 validated image filenames belonging to 42 classes.
1356/1356 [==============================] - 37228s 27s/step - loss: 1.2526 - accuracy: 0.7194 - val_loss: 14.4980 - val_accuracy: 0.0569
